In [173]:
import sys
sys.path.append('../')

import numpy as np

from helper import *

In [174]:
class minibatch:
    def mse_loss(self, x):
        y_pred = np.dot(self.X, x)
        mse = np.mean((self.y - y_pred) ** 2)
        return mse

    def mse_loss_grad(self, x):
        # print(self.X.shape, self.y.shape)

        # Choose n random data points from the training set without replacement
        indices = np.random.choice(self.X.shape[0], self.batch_size, replace=False)
        X_batch = self.X[indices, :]
        y_batch = self.y[indices]

        # print(X_batch.shape, y_batch.shape)

        # Compute the gradient of the MSE loss with respect to x for the chosen data points
        y_pred = np.dot(X_batch, x)
        grad = 2 * np.dot(X_batch.T, (y_pred - y_batch))

        # print(grad)

        # Sum values in rows of grad and divide by n
        # grad_mean = np.sum(grad, axis=1) / self.batch_size

        # print(grad_mean)
        return grad
    
    def __init__(self, X, y, batch_size=2, method='mse'):
        self.X = X
        self.y = y
        self.batch_size = batch_size

        if self.batch_size > X.shape[0]:
            self.batch_size = X.shape[0]

        if method == 'mse':
            self.f = self.mse_loss 
            self.grad = self.mse_loss_grad
        else:
            print('method not found')
        

    def constant_lr_scheduling(epoch, initial_lr):
        return initial_lr

    def gradient_descent(self, x0, lr_scheduling_func=constant_lr_scheduling, initial_lr=0.001, max_epochs=100, eps=1e-5, minimum = 0.0, apply_min=False, apply_value=True):
        """
        Cтохастический градиентный спуск для поиска минимума функции.

        Аргументы:
            x0 (list): Начальную точка, с которой начинается поиск.
            initial_lr (float): learning_rate - Начальная скорость обучения или шаг градиентного спуска.
            max_epochs (int): Максимальное количество эпох или итераций для выполнения алгоритма.
            minimum (float): Минимум функции.
            epsilon (float): Малое число, используемое как критерий останова для алгоритма.
        Возвращает:
            Список всех точек, посещенных во время алгоритма.
        """
        return custom_gradient_descent_with_lr_scheduling(self.f, self.grad, x0, lr_scheduling_func, initial_lr, max_epochs, eps, minimum, apply_min, apply_value)
    
    def get_loss_history(self, results):
        loss_history = []

        for i in range(len(results)):
            loss_history.append(self.f(results[i]))

        return loss_history
    


In [175]:
class get_model_LinearRegression:
    def __init__(self, batch_size=None):
        self.coef_ = []
        self.intercept_ = None
        self.batch_size = batch_size

    def fit(self, X_train, y_train, epsilon=24, learning_rate=0.001, max_epochs=100, batch_size=1, apply_min=True):
        X = X_train 
        y = y_train

        if self.batch_size is not None:
            batch_size = self.batch_size

        mb_gd = minibatch(X, y, batch_size=batch_size)
        x0 = np.zeros(X.shape[1], dtype=float)

        results = mb_gd.gradient_descent(x0, max_epochs=1000, initial_lr=1e-8, eps=70, apply_min=True, apply_value=True)

        self.coef_ = results[-1][:-1]
        self.intercept_ = results[-1][-1] 

    def predict(self, X_test):
        y_pred = []

        for i in range(len(X_test)):
            y = self.intercept_ + sum([self.coef_[j] * float(X_test[i][j]) for j in range(len(self.coef_))])
            y_pred.append(y)
        return y_pred

In [176]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import fetch_openml
from tqdm import tqdm

def get_reference_result(dataset):
    model = LinearRegression()
    return train_test_print_model(model, dataset, print_result=False, print_info=False, view_graphics=False)

def get_idea_result():
    return 0.0, 1.0

def research_LinearRegression_calculation(model_linear_regression):
    dataset = fetch_openml(name='boston', version=1, as_frame=True , parser='liac-arff')
    mse_reference, r2_reference = get_reference_result(dataset)
    mse_ideal, r2_ideal = get_idea_result()

    print(f"{'mse_reference:':<15}{mse_reference:<14.6g} {' r2_reference:':<15}{r2_reference:<14.6g}")
    print(f"{'mse_ideal:':<15}{mse_ideal:<14.6g} {' r2_ideal:':<15}{r2_ideal:<14.6g}")

    mse_results = []
    r2_results = []
    k = 10

    for i in range(1, dataset.data.values.shape[1] + 1):
        model = model_linear_regression(batch_size=i)
        mse = mse_reference/k
        r2 = r2_reference/k
        tries = 0
        # pbar = tqdm(desc=f"Batch size: {i}", unit=" test")
        while mse <= mse_reference/k or r2 <= r2_reference/k:    
            mse, r2 = train_test_print_model(model, dataset, print_result=False, view_graphics=False)
            tries += 1
            print(f"Batch size: {i}, Try: {tries}, MSE: {mse:.2f}, R^2: {r2:.2f}")
            # pbar.update()
        # pbar.close()

        mse_results.append(mse)
        r2_results.append(r2)
    
    return mse_results, r2_results, mse_reference, r2_reference, mse_ideal, r2_ideal

def research_LinearRegression_view(mse_results, r2_results, mse_reference, r2_reference, mse_ideal, r2_ideal):
    # Строим графики
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    # График MSE
    axs[0].plot(range(1, len(mse_results)+1), mse_results, marker='o')
    axs[0].set_xlabel('Batch size')
    axs[0].set_ylabel('MSE')
    axs[0].axhline(y=mse_reference, color='y', linestyle='--', label='MSE reference')
    axs[0].axhline(y=mse_ideal, color='g', linestyle='--', label='MSE ideal')
    axs[0].legend()

    # График R^2
    axs[1].plot(range(1, len(r2_results)+1), r2_results, marker='o')
    axs[1].set_xlabel('Batch size')
    axs[1].set_ylabel('R^2')
    axs[1].axhline(y=r2_reference, color='y', linestyle='--', label='R^2 reference')
    axs[1].axhline(y=r2_ideal, color='g', linestyle='--', label='R^2 ideal')
    axs[1].legend()

    plt.show()

def research_LinearRegression(model_linear_regression):
    mse_results, r2_results, mse_reference, r2_reference, mse_ideal, r2_ideal = research_LinearRegression_calculation(model_linear_regression)
    research_LinearRegression_view(mse_results, r2_results, mse_reference, r2_reference, mse_ideal, r2_ideal)

In [177]:
research_LinearRegression(get_model_LinearRegression)

mse_reference: 24.2911         r2_reference: 0.668759      
mse_ideal:     0               r2_ideal:     1             
Batch size: 1, Try: 1, MSE: 97.94, R^2: -0.34
Batch size: 1, Try: 2, MSE: 95.42, R^2: -0.30
Batch size: 1, Try: 3, MSE: 94.87, R^2: -0.29
Batch size: 1, Try: 4, MSE: 98.82, R^2: -0.35
Batch size: 1, Try: 5, MSE: 95.24, R^2: -0.30
Batch size: 1, Try: 6, MSE: 99.12, R^2: -0.35
Batch size: 1, Try: 7, MSE: 95.92, R^2: -0.31
Batch size: 1, Try: 8, MSE: 96.38, R^2: -0.31
Batch size: 1, Try: 9, MSE: 96.55, R^2: -0.32
Batch size: 1, Try: 10, MSE: 95.56, R^2: -0.30
Batch size: 1, Try: 11, MSE: 97.43, R^2: -0.33
Batch size: 1, Try: 12, MSE: 98.24, R^2: -0.34
Batch size: 1, Try: 13, MSE: 97.40, R^2: -0.33
Batch size: 1, Try: 14, MSE: 94.99, R^2: -0.30
Batch size: 1, Try: 15, MSE: 96.78, R^2: -0.32
Batch size: 1, Try: 16, MSE: 97.91, R^2: -0.34
Batch size: 1, Try: 17, MSE: 98.04, R^2: -0.34
Batch size: 1, Try: 18, MSE: 95.83, R^2: -0.31
Batch size: 1, Try: 19, MSE: 97.47, R^2: -0

KeyboardInterrupt: 